In [1]:
import requests
import googlemaps
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely as shp
from pyproj import Geod
from ast import literal_eval
import keyring
import json

In [2]:
API_KEY = keyring.get_password("goomaps", "yianzhang14")

In [3]:
gmaps = googlemaps.Client(key=API_KEY)

In [4]:
# somehwere in manhattan
loc = (40.73188892751728, -73.98927188478527)

In [5]:
df = pd.read_csv("data/urgent_cleaned.csv")
df

,Unnamed: 0,full address,NPI,Site Name,Address,Room/Suite,Town/City,CountyFIPS,County Name,State,...,Servicing County 4,Servicing County 5,OASAS Program Reporting Unit Number,OASAS Provider Number,OMH ID,Plan Name,Provider Network Data System Identification Number,Quarter/Year,geometry,type
0,2,"1 BROOKDALE PLZ RM LL91, BROOKLYN, NY 11212",1548217763,FLUSHING HOSPITAL MEDICAL CENTER,1 BROOKDALE PLZ,RM LL91,BROOKLYN,36047,Kings,NY,...,999,999,NaN,NaN,NaN,"UnitedHealthcare of New York, Inc.",148,Quarter 1 2021,POINT (307273.1349615707 54330.81319598604),NPI-2
1,4,"1 FORDHAM PLZ, BRONX, NY 10458",1720414154,MONTEFIORE NEW ROCHELLE HOSPITAL,1 FORDHAM PLZ,NaN,BRONX,36005,Bronx,NY,...,999,999,NaN,NaN,9999999999,Consolidated Health Plans: Cigna,168,Quarter 1 2021,POINT (309341.77659211366 77031.53626176476),NPI-2
2,6,"1 GUSTAVE L LEVY PL, NEW YORK, NY 10029",1013948595,MOUNT SINAI HOSPITAL,1 GUSTAVE L LEVY PL,NaN,NEW YORK,36061,New York,NY,...,999,999,1.000000e+10,1.000000e+10,6399020,"Oxford Health Plans (NY), Inc.: Metro",143,Quarter 1 2021,POINT (303913.30307026475 69277.7115768047),NPI-2
3,7,"1 PROSPECT PARK W, BROOKLYN, NY 11215",1043251390,PARK SLOPE EMERGENCY PHYS SVC,1 PROSPECT PARK W,NaN,BROOKLYN,36047,Kings,NY,...,999,999,NaN,NaN,NaN,"Oxford Health Plans (NY), Inc.: Metro",143,Quarter 1 2021,POINT (302481.54769885784 56164.37563754736),NPI-2
4,10,"100 E 77TH ST, NEW YORK, NY 10075",1316147176,LENOX HILL HOSPITAL,100 E 77TH ST,NaN,NEW YORK,36061,New York,NY,...,999,999,NaN,NaN,6649020,Jefferson-Lewis Et. Al. School Employees’ Heal...,212,Quarter 1 2021,POINT (303316.91374236497 67426.23223508784),NPI-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561,856,"9777 QUEENS BLVD, REGO PARK, NY 11374",1437230034,NYU HOSPITALS CENTER - CANCER CENTER,9777 QUEENS BLVD,NaN,REGO PARK,36081,Queens,NY,...,999,999,NaN,NaN,NaN,New York Quality Healthcare Corporation,44,Quarter 1 2021,POINT (311960.8099653642 62459.85496309886),NPI-2
562,857,"9920 4TH AVE STE 205, BROOKLYN, NY 11209",1205868528,SUNY DOWNSTATE MED CENTER,9920 4TH AVE,STE 205,BROOKLYN,36047,Kings,NY,...,999,999,NaN,NaN,NaN,"UnitedHealthcare of New York, Inc.",148,Quarter 1 2021,POINT (297167.59793859185 49596.59393249073),NPI-2
563,858,"JFK ARPT 14-140 TENANT #4 BLD, JAMAICA, NY 11430",1417349507,METRO URGENT MEDICAL CARE OF QUEENS,JFK ARPT 14-140 TENANT #4 BLD,NaN,JAMAICA,36081,Queens,NY,...,999,999,NaN,NaN,NaN,"Oxford Health Plans (NY), Inc.: Metro",143,Quarter 1 2021,POINT (316830.18732158607 54648.54333367746),NPI-2
564,860,"MADISON & 100 ST, NEW YORK, NY 10029",1255375986,MOUNT SINAI ELMHURST FCLTY PRAC G,MADISON & 100 ST,NaN,NEW YORK,36061,New York,NY,...,999,999,NaN,NaN,NaN,"UnitedHealthcare of New York, Inc.",148,Quarter 1 2021,POINT (304747.59900113015 69812.57038312979),NPI-2


In [6]:
def get_latlon(address):
    latlon = gmaps.geocode(address)[0]
    return ', '.join(str(x) for x in latlon["geometry"]["location"].values())

In [7]:
df["latlon"] = df["full address"].apply(get_latlon)

In [67]:
def get_place_details(row):
    res = {}
    res["status"] = "ok"
    res["opening_hours"] = ""
    res["place_id"] = ""
    res["gmaps_address"] = ""
    res["gmaps_name"] = ""
    res["gmaps_business_status"] = ""
    res["gmaps_lat"] = ""
    res["gmaps_lon"] = ""
    
    place_id = gmaps.places(query=row["Site Name"] + " " + row["full address"], location=row["latlon"], radius=64)
    if place_id["status"] != "OK":
        res["gmaps_status"] = "error - can't find place id"
        print("can't find place id", row["Site Name"])
        return res
    
    
    place = gmaps.place(place_id["results"][0]["place_id"])
    if place["status"] != "OK":
        print("can't match place id", row["Site Name"])
        res["gmaps_status"] = "error - can't match place id"
        return res
    place = place["result"]
    if "opening_hours" in place:
        res["opening_hours"] = json.dumps(place["opening_hours"])
    if "place_id" in place:
        res["place_id"] = place["place_id"]
        
    if "formatted_address" in place:
        res["gmaps_address"] = place["formatted_address"]
    
    if "name" in place:
        res["gmaps_name"] = place["name"]
        
    if "business_status" in place:
        res["gmaps_business_status"] = place["business_status"]
        
    if "geometry" in place:
        res["gmaps_lat"] = place["geometry"]["location"]["lat"]
        res["gmaps_lon"] = place["geometry"]["location"]["lng"]
        
    return res

In [68]:
applied_df = df.apply(lambda x: get_place_details(x), axis=1, result_type="expand")

can't find place id FLUSHING HOSPITAL MEDICAL CENTER
can't find place id 210 MJD MED PLLC DBA SILVER STAR UC
can't find place id FLUSHING HOSPITAL MEDICAL CENTER
can't find place id MMC EMERGENCY PHYSICIANS FPP
can't find place id NYU LANGONE MEDHATTAN
can't find place id MOUNT SINAI ELMHURST FCLTY PRAC G
can't find place id PENINSULA MEDICAL CARE PLLC
can't find place id MOUNT SINAI ELMHURST FCLTY PRAC G
can't find place id WMC CORNELL EMERGENCY PHYSICIANS
can't find place id TAKE CARE MED HLTH NY PC
can't find place id ROCHESTER GENERAL HOSP-PROF FEES
can't find place id MOUNT SINAI ELMHURST FCLTY PRAC G
can't find place id CITY MEDICAL OF UPPER EAST SIDE, PLLC DBA CITYMD U
can't find place id MOONLIGHT PEDIATRICS
can't find place id ESSEN MEDICAL URGICARE PLLC DBA METRO URGICARE PLL
can't find place id WMC CORNELL EMERGENCY PHYSICIANS
can't find place id BRONX LEBANON HSP/EM DEPT
can't find place id KINGSBROOK MEDICAL ASSOCIATES
can't find place id ESSEN MEDICAL URGICARE PLLC DBA ME

In [69]:
applied_df

,status,opening_hours,place_id,gmaps_status
0,ok,,,error - can't find place id
1,ok,"{""open_now"": true, ""periods"": [{""close"": {""day...",ChIJK5Kt9X_0wokRtSbTJb_VT5U,NaN
2,ok,,ChIJUyrmBQ9ZwokRJFt7Ck3MO1g,NaN
3,ok,,ChIJJ45eeghbwokRhTIYFdgy_tY,NaN
4,ok,"{""open_now"": true, ""periods"": [{""open"": {""day""...",ChIJra13dJVYwokRh3kQ5-p7Nio,NaN
...,...,...,...,...
561,ok,,ChIJ_4cGwTBewokR_-LKjhQKw2Y,NaN
562,ok,"{""open_now"": true, ""periods"": [{""open"": {""day""...",ChIJSaTDdllFwokR7WiY-lXXlBQ,NaN
563,ok,"{""open_now"": true, ""periods"": [{""close"": {""day...",ChIJk_ZCRAhnwokRPXEE2Va4IQE,NaN
564,ok,,ChIJT1R9W6dYwokRleZSU1oGeCA,NaN


In [70]:
df_combined = pd.concat([df, applied_df], axis=1)

In [71]:
df_combined.to_csv("data/urgent_time.csv")

In [72]:
(df_combined["opening_hours"] == "").sum()

186

In [ ]:
df_combined.to_csv("data/urgent_time.csv")

In [34]:
# df_time = pd.read_csv("data/urgent_time.csv")

# df_time["gmaps_address"] = ""
# df_time["gmaps_name"] = ""
# df_time["gmaps_business_status"] = ""
# df_time["gmaps_lat"] = -1
# df_time["gmaps_lon"] = -1

# for i, row in df_time.iterrows():
#     if pd.isna(row["place_id"]):
#        continue
   
#     curr = gmaps.place(row["place_id"])
    
#     if curr["status"] != "OK":
#         print(f"error in row {i}")
#         continue
    
#     curr = curr["result"]
    
#     if "formatted_address" in curr:
#         df_time.loc[i, "gmaps_address"] = curr["formatted_address"]
    
#     if "name" in curr:
#         df_time.loc[i, "gmaps_name"] = curr["name"]
        
#     if "business_status" in curr:
#         df_time.loc[i, "gmaps_business_status"] = curr["business_status"]
        
#     if "geometry" in curr:
#         df_time.loc[i, "gmaps_lat"] = curr["geometry"]["location"]["lat"]
#         df_time.loc[i, "gmaps_lon"] = curr["geometry"]["location"]["lng"]
        
# df_time.to_csv("data/urgent_time.csv")

,Unnamed: 0.1,Unnamed: 0,full address,NPI,Site Name,Address,Room/Suite,Town/City,CountyFIPS,County Name,...,Plan Name,Provider Network Data System Identification Number,Quarter/Year,geometry,type,latlon,status,opening_hours,place_id,gmaps_status
0,0,2,"1 BROOKDALE PLZ RM LL91, BROOKLYN, NY 11212",1548217763,FLUSHING HOSPITAL MEDICAL CENTER,1 BROOKDALE PLZ,RM LL91,BROOKLYN,36047,Kings,...,"UnitedHealthcare of New York, Inc.",148,Quarter 1 2021,POINT (307273.1349615707 54330.81319598604),NPI-2,"40.6549132, -73.9126971",ok,NaN,NaN,error - can't find place id
1,1,4,"1 FORDHAM PLZ, BRONX, NY 10458",1720414154,MONTEFIORE NEW ROCHELLE HOSPITAL,1 FORDHAM PLZ,NaN,BRONX,36005,Bronx,...,Consolidated Health Plans: Cigna,168,Quarter 1 2021,POINT (309341.77659211366 77031.53626176476),NPI-2,"40.8603554, -73.8899228",ok,"{""open_now"": true, ""periods"": [{""close"": {""day...",ChIJK5Kt9X_0wokRtSbTJb_VT5U,NaN
2,2,6,"1 GUSTAVE L LEVY PL, NEW YORK, NY 10029",1013948595,MOUNT SINAI HOSPITAL,1 GUSTAVE L LEVY PL,NaN,NEW YORK,36061,New York,...,"Oxford Health Plans (NY), Inc.: Metro",143,Quarter 1 2021,POINT (303913.30307026475 69277.7115768047),NPI-2,"40.79001179999999, -73.95374140000001",ok,NaN,ChIJUyrmBQ9ZwokRJFt7Ck3MO1g,NaN
3,3,7,"1 PROSPECT PARK W, BROOKLYN, NY 11215",1043251390,PARK SLOPE EMERGENCY PHYS SVC,1 PROSPECT PARK W,NaN,BROOKLYN,36047,Kings,...,"Oxford Health Plans (NY), Inc.: Metro",143,Quarter 1 2021,POINT (302481.54769885784 56164.37563754736),NPI-2,"40.6694299, -73.9728265",ok,NaN,ChIJJ45eeghbwokRhTIYFdgy_tY,NaN
4,4,10,"100 E 77TH ST, NEW YORK, NY 10075",1316147176,LENOX HILL HOSPITAL,100 E 77TH ST,NaN,NEW YORK,36061,New York,...,Jefferson-Lewis Et. Al. School Employees’ Heal...,212,Quarter 1 2021,POINT (303316.91374236497 67426.23223508784),NPI-2,"40.77373960000001, -73.96073779999999",ok,"{""open_now"": true, ""periods"": [{""open"": {""day""...",ChIJra13dJVYwokRh3kQ5-p7Nio,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561,561,856,"9777 QUEENS BLVD, REGO PARK, NY 11374",1437230034,NYU HOSPITALS CENTER - CANCER CENTER,9777 QUEENS BLVD,NaN,REGO PARK,36081,Queens,...,New York Quality Healthcare Corporation,44,Quarter 1 2021,POINT (311960.8099653642 62459.85496309886),NPI-2,"40.7292451, -73.8582167",ok,NaN,ChIJ_4cGwTBewokR_-LKjhQKw2Y,NaN
562,562,857,"9920 4TH AVE STE 205, BROOKLYN, NY 11209",1205868528,SUNY DOWNSTATE MED CENTER,9920 4TH AVE,STE 205,BROOKLYN,36047,Kings,...,"UnitedHealthcare of New York, Inc.",148,Quarter 1 2021,POINT (297167.59793859185 49596.59393249073),NPI-2,"40.613221, -74.03392099999999",ok,"{""open_now"": true, ""periods"": [{""open"": {""day""...",ChIJSaTDdllFwokR7WiY-lXXlBQ,NaN
563,563,858,"JFK ARPT 14-140 TENANT #4 BLD, JAMAICA, NY 11430",1417349507,METRO URGENT MEDICAL CARE OF QUEENS,JFK ARPT 14-140 TENANT #4 BLD,NaN,JAMAICA,36081,Queens,...,"Oxford Health Plans (NY), Inc.: Metro",143,Quarter 1 2021,POINT (316830.18732158607 54648.54333367746),NPI-2,"40.6586769, -73.7953211",ok,"{""open_now"": true, ""periods"": [{""close"": {""day...",ChIJk_ZCRAhnwokRPXEE2Va4IQE,NaN
564,564,860,"MADISON & 100 ST, NEW YORK, NY 10029",1255375986,MOUNT SINAI ELMHURST FCLTY PRAC G,MADISON & 100 ST,NaN,NEW YORK,36061,New York,...,"UnitedHealthcare of New York, Inc.",148,Quarter 1 2021,POINT (304747.59900113015 69812.57038312979),NPI-2,"40.7868236, -73.9452414",ok,NaN,ChIJT1R9W6dYwokRleZSU1oGeCA,NaN


In [170]:
query = "Urgent Care Far Rockaway | Far Rockaway Urgent Care | Urgent Care 11691"

id = gmaps.places(query, location=loc)["results"][0]["place_id"]
print(id)

x = gmaps.place(id)
if "opening_hours" in x["result"]:
    print(x["result"]["opening_hours"])
print()
print(x["result"]["formatted_address"])
print()
print(x["result"]["name"])
print()
print(x["result"]["business_status"])
print()

ChIJq0592ihpwokRGwP5-EFDGOk
{'open_now': True, 'periods': [{'close': {'day': 1, 'time': '1900'}, 'open': {'day': 1, 'time': '1000'}}, {'close': {'day': 2, 'time': '1900'}, 'open': {'day': 2, 'time': '1000'}}, {'close': {'day': 3, 'time': '1900'}, 'open': {'day': 3, 'time': '1000'}}, {'close': {'day': 4, 'time': '1900'}, 'open': {'day': 4, 'time': '1000'}}, {'close': {'day': 5, 'time': '1900'}, 'open': {'day': 5, 'time': '1000'}}], 'weekday_text': ['Monday: 10:00\u202fAM\u2009–\u20097:00\u202fPM', 'Tuesday: 10:00\u202fAM\u2009–\u20097:00\u202fPM', 'Wednesday: 10:00\u202fAM\u2009–\u20097:00\u202fPM', 'Thursday: 10:00\u202fAM\u2009–\u20097:00\u202fPM', 'Friday: 10:00\u202fAM\u2009–\u20097:00\u202fPM', 'Saturday: Closed', 'Sunday: Closed']}

228 Beach 20th St, Queens, NY 11691, USA

Urgent Care Far Rockaway | Far Rockaway Urgent Care | Urgent Care 11691

OPERATIONAL

